Ref: https://www.youtube.com/watch?v=yMk_XtIEzH8&list=PLQVvvaa0QuDezJFIOU5wDdfy4e9vdnx-7
https://www.youtube.com/watch?v=Gq1Azv_B4-4&list=PLQVvvaa0QuDezJFIOU5wDdfy4e9vdnx-7&index=2
https://code.visualstudio.com/docs/python/jupyter-support


In [82]:
!pip  install gym

In [83]:
import gym
import numpy as np

In [84]:
env = gym.make("MountainCar-v0")
#three action it can take

env.reset()
done = False

while not done:
    action = 2
    new_state, reward, done, _ =env.step(action)
    #when we initialize with q values its random values
    #agent will explore a lot
    print(new_state)
    print(reward)
    #reward always -1, reaches the flag = 0
    
    #new_state = state sensing from the environment - ex: pos, vel in this case
    env.render(mode = "ipython") #show it
  
#action 0: do nothing
#action 1: push the car left
#action 2: push the car right
env.close()


[-4.41486396e-01  3.93227311e-04]
-1.0
[-0.4407028   0.00078359]
-1.0
[-0.43953454  0.00116826]
-1.0
[-0.4379901   0.00154444]
-1.0
[-0.43608068  0.00190941]
-1.0
[-0.43382014  0.00226054]
-1.0
[-0.43122483  0.00259531]
-1.0
[-0.42831348  0.00291134]
-1.0
[-0.42510709  0.0032064 ]
-1.0
[-0.42162868  0.00347841]
-1.0
[-0.41790317  0.00372551]
-1.0
[-0.41395716  0.00394601]
-1.0
[-0.40981872  0.00413844]
-1.0
[-0.40551715  0.00430157]
-1.0
[-0.40108279  0.00443436]
-1.0
[-0.39654675  0.00453604]
-1.0
[-0.39194069  0.00460606]
-1.0
[-0.38729659  0.0046441 ]
-1.0
[-0.38264651  0.00465008]
-1.0
[-0.37802235  0.00462416]
-1.0
[-0.37345563  0.00456672]
-1.0
[-0.3689773   0.00447834]
-1.0
[-0.36461748  0.00435982]
-1.0
[-0.36040533  0.00421215]
-1.0
[-0.35636882  0.00403651]
-1.0
[-0.35253457  0.00383424]
-1.0
[-0.34892773  0.00360684]
-1.0
[-0.34557179  0.00335594]
-1.0
[-0.34248847  0.00308332]
-1.0
[-0.33969763  0.00279084]
-1.0
[-0.33721713  0.0024805 ]
-1.0
[-0.33506279  0.00215434]
-1.0


Many random values for new_state, we will not include all the numbers in the Q table
So we will convert these continuous values to discrete values

# Build the Q table

In [85]:
print(env.observation_space.high)
print(env.observation_space.low)
print(env.action_space.n)


[0.6  0.07]
[-1.2  -0.07]
3


Sometimes we don't know the values, so we have to work randomly then

In [86]:
#managable size for Q-table, size changes according to env
DISCRETE_OS_SIZE = [20]*len(env.observation_space.high)
print(DISCRESTE_OS_SIZE)

[20, 20]


In [87]:
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRESTE_OS_SIZE
print(discrete_os_win_size)

[0.09  0.007]


In [88]:
q_table = np.random.uniform(low = 2, high = 0, size = (DISCRETE_OS_SIZE + [env.action_space.n]))
#every possible combo*3 - random values, later will be optimized
print(q_table.shape)

(20, 20, 3)


In [89]:
LEARNING_RATE = 0.1 #0-1
DISCOUNT = 0.95 #WEIGHT - MEASURE OF IMPORTANCE OF FUTURE ACTIONS OVER CURRENT ACTIONS
EPISODES = 25000


In [90]:
def get_discrete_state(state):
    discrete_state = (state - env.observation_space.low) / discrete_os_win_size
    return tuple(discrete_state.astype(np.int))

In [91]:
discrete_state = get_discrete_state(env.reset())
#reset gives the initial value of state
print(discrete_state)

(8, 10)


In [92]:
print(q_table[discrete_state])

[0.06661234 0.15727543 0.92576308]


In [93]:
#if we want to check the highest value of the action
print(np.argmax(q_table[discrete_state]))


2


In this case the returned index is 0, so action no 0

In [94]:
SHOW_EVERY = 2000

In [95]:
done = False
for episode in range(EPISODES):
    if episode %SHOW_EVERY == 0:
        render = True
    else:
        render = False

    while not done:
        action = np.argmax(q_table[discrete_state])
        new_state, reward, done, _ =env.step(action)
        new_discrete_state = get_discrete_state(new_state)
        if render:
            env.render(mode = "rgb-array") #show it
        if not done:
            max_future_q = np.max(q_table[new_discrete_state])
            current_q = q_table[discrete_state + (action, )]

            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            q_table[discrete_state + (action, )] = new_q
        elif new_state[0]>=env.goal_position:
            q_table[discrete_state + (action, )] = 0
        
        discrete_state = new_discrete_state
env.close()
